"# 🧠 Agente LangChain con herramientas personalizadas\n",
    "\n",
    "Este notebook implementa un agente en LangChain v0.3 usando herramientas personalizadas con el modelo GPT-4 y la arquitectura `Runnable`. Está basado en la nueva sintaxis modular de LangChain 0.3."


"## 1. 📦 Importación de librerías\n",
    "Importamos los módulos necesarios para construir el agente, sus herramientas y la cadena de ejecución."

In [2]:
from langchain_core.runnables import Runnable
from langchain_openai import ChatOpenAI
from langchain.agents import Tool, AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents.format_scratchpad.openai_tools import format_to_openai_tool_messages
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain.tools import tool

"## 2. 🔑 Lectura segura de la clave de OpenAI\n",
    "Guardamos la clave en un archivo externo `OpenAI_key.txt` para evitar exponerla directamente."

In [ ]:
with open('..\OpenAI_key.txt') as f:
    api_key = f.read()

"## 3. 🧠 Inicialización del modelo de lenguaje (LLM)\n",
    "Creamos una instancia del modelo `gpt-4` para poder utilizar herramientas mediante `tool calling`. Se fija `temperature=0` para hacerlo más determinista."

In [26]:
llm = ChatOpenAI(openai_api_key=api_key, temperature=0, model="gpt-4")

"## 4. 🛠️ Definición de herramientas personalizadas\n",
    "A continuación, definimos dos herramientas con el decorador `@tool`, que permite que LangChain las reconozca automáticamente."

In [15]:
@tool
def persona_amable(text: str) -> str:
    "Retorna la persona más amable del universo. No se espera entrada"
    return "Miguel Celebres"

@tool
def llm_math(expression: str) -> str:
    "Evalúa una expresión matemática simple y devuelve el resultado."
    try:
        return str(eval(expression))
    except Exception as e:
        return f"Error al evaluar la expresión: {str(e)}"

## 🔍 Inclusión de la herramienta Wikipedia\n",
    "Esta herramienta permite consultar información directamente desde Wikipedia usando el wrapper oficial de LangChain."

In [17]:
# Cargar herramientas estándar\n",
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain.agents import Tool

In [19]:
#Definir las herramientas
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

In [20]:
tools = [
    Tool.from_function(func=wikipedia.run, name="wikipedia", description="Busca en Wikipedia"),
    Tool.from_function(func=llm_math, name="llm-math", description="Evalúa operaciones matemáticas simples"),
    Tool.from_function(func=persona_amable, name="persona_amable", description="Retorna la persona más amable")
    ]

"## 5. 🧾 Definición del prompt del agente\n",
    "Usamos `ChatPromptTemplate` para estructurar la conversación entre el usuario y el agente. El campo `agent_scratchpad` se utiliza para registrar razonamientos intermedios del agente durante la ejecución."
 

In [22]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Eres un agente útil que responde utilizando herramientas si es necesario."),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")])

"## 6. 🔗 Creación de la cadena `Runnable` del agente\n",
    "Aquí combinamos el procesamiento de entrada, el prompt, el modelo y el parser de salida en una única cadena ejecutable."

In [27]:
agent_runnable = (
    {
    "input": lambda x: x["input"],
    "agent_scratchpad": lambda x: format_to_openai_tool_messages(x["intermediate_steps"])
        }
    | prompt
    | llm
    | OpenAIToolsAgentOutputParser()
    )

"## 8. 🤖 Construcción del agente ejecutor (`AgentExecutor`)\n",
    "`AgentExecutor` se encarga de gestionar la ejecución completa del agente, incluyendo la llamada al modelo, el manejo de herramientas y la visualización en modo `verbose`."

In [ ]:
agent_executor = AgentExecutor(agent=agent_runnable, tools=tools, verbose=True)

## 9. 🧪 Ejecución de una consulta\n",
    "Finalmente, probamos al agente con una entrada que requiere el uso de la herramienta `llm_math`. El agente debe razonar, decidir llamar a la herramienta y mostrar el resultado."

In [ ]:
response = agent_executor.invoke({"input": "¿Quién es la persona más amable del universo?"})
print(response)